In [1]:
!pip3 install -q --upgrade pip
!pip3 install -q pandas numpy matplotlib seaborn openpyxl climateserv requests netCDF4 xarray pyproj python-dotenv

In [2]:
import sys
from pathlib import Path

# Calculate the path to the scripts folder relative to the current notebook.
scripts_dir = Path("../../").resolve()

# Add the scripts directory to the sys.path if it's not already there.
if str(scripts_dir) not in sys.path:
    sys.path.append(str(scripts_dir))

import os
import time
import json
import requests
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
from dotenv import load_dotenv
from scipy.spatial import KDTree
from shapely.geometry import Point
from typing import Dict, List, Tuple, Optional, Any
# Load environment variables from .env file
load_dotenv()

True

In [6]:
import pandas as pd

# Read the CSV file into a dataframe
df = pd.read_csv('../../data/final_dataset/processed_data/merged_data.csv')

df_appears = pd.read_csv('../../data/final_dataset/processed_data/appears/appears_ndvi_mir_evi.csv')
df_appears.drop(['_250m_16_days_NIR_reflectance'], axis=1, inplace=True)

In [7]:
df

,Date,long,lat,Drainage_density,Slope_tif2,Hydrogeo,Curvatu_tif2,NASADEM_HGT,DepthToGroundwater,NASA_IMERG_Late,LIS_ET,LIS_Soil_Moisture_Combined,LIS_Streamflow
0,12-01-2023,-16.670616,13.488346,13.499496,0.000000,5.0,-0.000000e+00,0.0,0-7,0.0,1.766242,0.209369,0.179610
1,12-01-2023,-16.665998,13.488376,14.573664,0.000000,5.0,-0.000000e+00,0.0,0-7,0.0,1.766242,0.209369,0.179610
2,12-01-2023,-16.684438,13.483734,15.609715,0.000000,5.0,-0.000000e+00,0.0,0-7,0.0,1.252523,0.220998,0.170694
3,12-01-2023,-16.679820,13.483765,13.571120,0.000000,5.0,-0.000000e+00,0.0,0-7,0.0,1.766242,0.209369,0.179610
4,12-01-2023,-16.675202,13.483796,13.922873,0.000000,5.0,-0.000000e+00,0.0,0-7,0.0,1.766242,0.209369,0.179610
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869,12-01-2023,-16.746177,13.076569,34.977886,1.642019,5.0,2.592000e+09,9.0,0-7,0.0,2.358303,0.295715,0.001112
2870,12-01-2023,-16.741567,13.076600,44.632805,2.321531,5.0,-5.184000e+09,2.0,0-7,0.0,2.358303,0.295715,0.001112
2871,12-01-2023,-16.736957,13.076631,49.922588,2.706814,5.0,3.888000e+09,2.0,0-7,0.0,2.000822,0.274988,0.106376
2872,12-01-2023,-16.732347,13.076662,49.457001,0.734494,5.0,-3.888000e+09,3.0,0-7,0.0,2.000822,0.274988,0.106376


In [8]:
df_appears

,latitude,longitude,_250m_16_days_EVI,_250m_16_days_MIR_reflectance,_250m_16_days_NDVI
0,13.611458,-16.559894,0.3257,0.0285,0.8015
1,13.609375,-16.559748,0.2549,0.0285,0.7009
2,13.609375,-16.557604,0.3017,0.0285,0.7627
3,13.609375,-16.555461,0.2998,0.0415,0.7746
4,13.609375,-16.553317,0.2818,0.0415,0.6967
...,...,...,...,...,...
104664,13.073958,-16.747660,0.3321,0.1650,0.5587
104665,13.071875,-16.753935,0.0344,0.1650,0.1055
104666,13.071875,-16.751796,0.0344,0.1650,0.1055
104667,13.071875,-16.749657,0.2304,0.1650,0.4039


In [13]:
from scipy.spatial import KDTree
import pandas as pd

def find_nearest(df_base: pd.DataFrame, df_to_merge: pd.DataFrame, columns_to_add: list) -> pd.DataFrame:
    """
    Encuentra el punto más cercano en df_to_merge para cada fila en df_base y fusiona los datos especificados.
    """
    print("Creando árbol KD...")
    tree = KDTree(df_to_merge[['latitude', 'longitude']])

    print("Buscando vecinos más cercanos...")
    # Cambio aquí: tree.query devuelve una tupla (distancias, índices)
    _, indices = tree.query(df_base[['lat', 'long']])

    print("Fusionando columnas...")
    for col in columns_to_add:
        print(f"Agregando columna: {col}")
        # Cambio aquí:
        # Usar 'iloc' con los índices obtenidos para asignar los valores
        df_base[col] = df_to_merge.iloc[indices][col].values
        print(f"Columna {col} agregada exitosamente.")

    print("Proceso completado.")
    return df_base

# Columnas para agregar de df_appears a df
columns_to_add = ['_250m_16_days_EVI', '_250m_16_days_MIR_reflectance', '_250m_16_days_NDVI']


In [14]:
# Llamar a la función y realizar la fusión
df_merged = find_nearest(df, df_appears, columns_to_add)

Creando árbol KD...
Buscando vecinos más cercanos...
Fusionando columnas...
Agregando columna: _250m_16_days_EVI
Columna _250m_16_days_EVI agregada exitosamente.
Agregando columna: _250m_16_days_MIR_reflectance
Columna _250m_16_days_MIR_reflectance agregada exitosamente.
Agregando columna: _250m_16_days_NDVI
Columna _250m_16_days_NDVI agregada exitosamente.
Proceso completado.


In [15]:
df_merged

,Date,long,lat,Drainage_density,Slope_tif2,Hydrogeo,Curvatu_tif2,NASADEM_HGT,DepthToGroundwater,NASA_IMERG_Late,LIS_ET,LIS_Soil_Moisture_Combined,LIS_Streamflow,_250m_16_days_EVI,_250m_16_days_MIR_reflectance,_250m_16_days_NDVI
0,12-01-2023,-16.670616,13.488346,13.499496,0.000000,5.0,-0.000000e+00,0.0,0-7,0.0,1.766242,0.209369,0.179610,0.0159,0.0145,0.1070
1,12-01-2023,-16.665998,13.488376,14.573664,0.000000,5.0,-0.000000e+00,0.0,0-7,0.0,1.766242,0.209369,0.179610,0.0692,0.0277,0.1814
2,12-01-2023,-16.684438,13.483734,15.609715,0.000000,5.0,-0.000000e+00,0.0,0-7,0.0,1.252523,0.220998,0.170694,-0.0034,0.0120,-0.0985
3,12-01-2023,-16.679820,13.483765,13.571120,0.000000,5.0,-0.000000e+00,0.0,0-7,0.0,1.766242,0.209369,0.179610,0.0119,0.0162,0.1739
4,12-01-2023,-16.675202,13.483796,13.922873,0.000000,5.0,-0.000000e+00,0.0,0-7,0.0,1.766242,0.209369,0.179610,0.0289,0.0873,0.1759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869,12-01-2023,-16.746177,13.076569,34.977886,1.642019,5.0,2.592000e+09,9.0,0-7,0.0,2.358303,0.295715,0.001112,0.2918,0.0619,0.5963
2870,12-01-2023,-16.741567,13.076600,44.632805,2.321531,5.0,-5.184000e+09,2.0,0-7,0.0,2.358303,0.295715,0.001112,0.2746,0.0619,0.6281
2871,12-01-2023,-16.736957,13.076631,49.922588,2.706814,5.0,3.888000e+09,2.0,0-7,0.0,2.000822,0.274988,0.106376,0.3266,0.0475,0.5873
2872,12-01-2023,-16.732347,13.076662,49.457001,0.734494,5.0,-3.888000e+09,3.0,0-7,0.0,2.000822,0.274988,0.106376,0.3663,0.0475,0.6597


In [17]:
percentage_of_nans = df_merged.isnull().mean() * 100
percentage_of_nans


Date                             0.0
long                             0.0
lat                              0.0
Drainage_density                 0.0
Slope_tif2                       0.0
Hydrogeo                         0.0
Curvatu_tif2                     0.0
NASADEM_HGT                      0.0
DepthToGroundwater               0.0
NASA_IMERG_Late                  0.0
LIS_ET                           0.0
LIS_Soil_Moisture_Combined       0.0
LIS_Streamflow                   0.0
_250m_16_days_EVI                0.0
_250m_16_days_MIR_reflectance    0.0
_250m_16_days_NDVI               0.0
dtype: float64

In [18]:
df_merged.rename(columns={'_250m_16_days_EVI': 'MOD13Q1_061__250m_16_days_EVI',
                          '_250m_16_days_MIR_reflectance': 'MOD13Q1_061__250m_16_days_MIR_reflectance',
                          '_250m_16_days_NDVI': 'MOD13Q1_061__250m_16_days_NDVI'}, inplace=True)


In [20]:
df_merged.to_csv('../../data/final_dataset/processed_data/merged_data.csv', index=False)
